# Modèle from scratch : classification binaire COVID / SAIN (NORMAL)

In [1]:
# Chargement des modules necessaire :
import os

import numpy as np

import random
import matplotlib as plt

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img,img_to_array

In [3]:
# On declare les chemins vers les donnees :

Image = 'data'
        
train_data_dir = 'Data/TRAIN'
validation_data_dir = 'Data/VALIDATION'
test_data_dir = 'Data/TEST'

# Dimmension et path :
img_width, img_height = 224, 224

ImageTRAINCOVID = os.listdir(Image + '/TRAIN/COVID')
ImageTRAINNORMAL = os.listdir(Image + '/TRAIN/NORMAL')

ImageTESTCOVID = os.listdir(Image + '/TEST/COVID')
ImageTESTNORMAL = os.listdir(Image + '/TEST/NORMAL')

ImageVALIDATIONCOVID = os.listdir(Image + '/VALIDATION/COVID')
ImageVALIDATIONNORMAL = os.listdir(Image + '/VALIDATION/NORMAL')

print('Il y a ' + str(len(ImageTRAINCOVID)) + ' images d\'entrainement de patient covid.') 
print('Il y a ' + str(len(ImageTRAINNORMAL)) + ' images d\'entrainement de patient non-covid.') 
print('Il y a ' + str(len(ImageTESTCOVID)) + ' images test de patient covid.') 
print('Il y a ' + str(len(ImageTESTNORMAL)) + ' images test de patient non-covid.') 
print('Il y a ' + str(len(ImageVALIDATIONCOVID)) + ' images de validation de patient covid.') 
print('Il y a ' + str(len(ImageVALIDATIONNORMAL)) + ' images de validation de patient non-covid.') 

Il y a 401 images d'entrainement de patient covid.
Il y a 401 images d'entrainement de patient non-covid.
Il y a 201 images test de patient covid.
Il y a 1001 images test de patient non-covid.
Il y a 101 images de validation de patient covid.
Il y a 101 images de validation de patient non-covid.


In [ ]:
# Visualisation de 6 exemples en 2 lignes et 3 colonnes pour chaque classe :

# TRAIN :

print("TRAIN COVID")


# COVID  :

plt.figure(figsize=(20,10))
for i in range(3):
    plt.subplot(1, 3, i + 1)
    randomImageTRAINCOVID = random.choice(os.listdir(train_data_dir + '/COVID'))
    plt.imshow(plt.imread(os.path.join( train_data_dir +'/COVID',randomImageTRAINCOVID)), cmap='gray')
    plt.title(randomImageTRAINCOVID)
    
plt.show()

print("TEST COVID")
# COVID  :
plt.figure(figsize=(20,10))
for i in range(3):
    plt.subplot(1, 3, i + 1)
    randomImageTESTCOVID = random.choice(os.listdir(validation_data_dir + '/COVID'))
    plt.imshow(plt.imread(os.path.join( validation_data_dir +'/COVID',randomImageTESTCOVID)), cmap='gray')
    plt.title(randomImageTESTCOVID)
    
plt.show()

print("TRAIN NORMAL")
# NORMAL :

plt.figure(figsize=(20,10))
for i in range(3):
    plt.subplot(1, 3, i + 1)
    randomImageTRAINNORMAL = random.choice(os.listdir(train_data_dir + '/NORMAL'))
    plt.imshow(plt.imread(os.path.join( train_data_dir +'/NORMAL',randomImageTRAINNORMAL)), cmap='gray')
    plt.title(randomImageTRAINNORMAL)
    
plt.show()


# TEST:

print("TEST NORMAL")

# NORMAL :

plt.figure(figsize=(20,10))
for i in range(3):
    plt.subplot(1, 3, i + 1)
    randomImageTESTNORMAL = random.choice(os.listdir(validation_data_dir + '/NORMAL'))
    plt.imshow(plt.imread(os.path.join( validation_data_dir +'/NORMAL',randomImageTESTNORMAL)), cmap='gray')
    plt.title(randomImageTESTNORMAL)
    
plt.show()

In [14]:
# Preprocessing :

# On rescale les images :

datagen = ImageDataGenerator(rescale=1./255)

# On definit la batch size :

batch_size = 32 

# On prepare les tableaux de donnees depuis les images :

train_generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='binary')

test_generator = datagen.flow_from_directory(
        test_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='binary')

validation_generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='binary')

Found 160 images belonging to 2 classes.
Found 1200 images belonging to 2 classes.
Found 1200 images belonging to 2 classes.


In [4]:
# On definit l'architecture du model :

model = Sequential()
model.add(Convolution2D(32, (3, 3), input_shape=(img_width, img_height,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

# On compile le modeles :

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# On affiche le model :

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 222, 222, 32)      896       
_________________________________________________________________
activation (Activation)      (None, 222, 222, 32)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 111, 111, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 109, 109, 32)      9248      
_________________________________________________________________
activation_1 (Activation)    (None, 109, 109, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 54, 54, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 52, 52, 64)       

In [ ]:
# On definit les paramétres pour l'entrainement :

epochs = 4
train_samples = 802
validation_samples = 202
test_samples = 1201

In [ ]:
# Entrainement du modele :

model.fit_generator(
        train_generator,
        steps_per_epoch=train_samples // batch_size,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=validation_samples// batch_size)

In [12]:
def testImage(path):
    img = load_img(path, target_size=(224,224))
    img = img_to_array(img)
    img = img.reshape((1, img.shape[0], img.shape[1], img.shape[2]))
    y = model.predict(img)
    return y[0][0]

In [ ]:
# On evalue le modele sur les donnes de validation :

resultValidation = model.evaluate_generator(validation_generator, validation_samples)

# On evalue le modele sur les donnes de test :

resultTest = model.evaluate_generator(test_generator, test_samples)


# On test quelque image aleatoirement choisis :

resultExemple = testImage("Data/EXEMPLE/COVID1")

    # Visualisation de l'image puis test puis affichage du resulat
    
print(resultValidation)
print(resultTest)
print(resultExemple)

In [ ]:
# On enregistre l'historique et les poids :

model.save_weights('models/scratch_10G_V1.h5')

In [5]:
# On instencie notre objet ImageDataGenerator

MonPetitGenerateur = ImageDataGenerator(
        rotation_range=20,
        zoom_range=0.15,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.15,
        horizontal_flip=True,
        fill_mode="nearest", cval=125)

In [6]:
# On stock les images que l'on veut augmenter

image_directory = 'Data/TO-AUGMENTED/COVID/'
image_directory2 = 'Data/TO-AUGMENTED/NORMAL/'
SIZE = 500
datasetCovid = []
datasetNormal = []

my_images = os.listdir(image_directory)
for i, image_name in enumerate(my_images):
    if (image_name.split('.')[1] == 'jpg'):
        image = load_img(image_directory + image_name,target_size = (224,224))
        image = img_to_array(image)
        image = image.reshape((image.shape[0], image.shape[1], image.shape[2]))
        datasetCovid.append(image)
x = np.array(datasetCovid)

my_images2 = os.listdir(image_directory2)
for i, image_name in enumerate(my_images2):
    if (image_name.split('.')[1] == 'jpg'):
        image2 = load_img(image_directory2 + image_name,target_size = (224,224))
        image2 = img_to_array(image2)
        image2 = image2.reshape((image2.shape[0], image2.shape[1], image2.shape[2]))
        datasetNormal.append(image2)
y = np.array(datasetNormal)


In [ ]:
# On augmente et on stock les nouvelles images

i = 0
for batch in MonPetitGenerateur.flow(x, batch_size=32,  
                          save_to_dir='Data/AUGMENTED/COVID-AUGMENTED', 
                          save_prefix='aug', 
                          save_format='png'):
    i += 1
    if i > 19:
        break

In [ ]:
i = 0
for batch in MonPetitGenerateur.flow(y, batch_size=32,  
                          save_to_dir='Data/AUGMENTED/NORMAL-AUGMENTED', 
                          save_prefix='aug', 
                          save_format='png'):
    i += 1
    if i > 19:
        break

In [7]:
# On declare les chemins vers les nouvelles donnees :
Image = 'data'
train_data_dir = 'Data/AUGMENTED'
validation_data_dir = 'Data/TEST'

# On declare les dimensions pour les images (224,224) :

img_width, img_height = 224, 224

# On liste et stocke les chemins des images :

ImageTRAINCOVID = os.listdir(Image + '/AUGMENTED/COVID-AUGMENTED')
ImageTRAINNORMAL = os.listdir(Image + '/AUGMENTED/NORMAL-AUGMENTED')

ImageTESTCOVID = os.listdir(Image + '/TEST/COVID')
ImageTESTNORMAL = os.listdir(Image + '/TEST/NORMAL')

# On affiche le nombre d'image trouve :

print('Il y a ' + str(len(ImageTRAINCOVID)) + ' images d\'entrainement de patient covid.') 
print('Il y a ' + str(len(ImageTRAINNORMAL)) + ' images d\'entrainement de patient non-covid.') 
print('Il y a ' + str(len(ImageTESTCOVID)) + ' images test de patient covid.') 
print('Il y a ' + str(len(ImageTESTNORMAL)) + ' images test de patient non-covid.') 


Il y a 80 images d'entrainement de patient covid.
Il y a 80 images d'entrainement de patient non-covid.
Il y a 201 images test de patient covid.
Il y a 1001 images test de patient non-covid.


In [15]:
# On rescale les images :

datagen = ImageDataGenerator(rescale=1./255)

# On definit la batch size :

batch_size = 32 

# On prepare les tableaux de donnees depuis les images :

train_generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='binary')

test_generator = datagen.flow_from_directory(
        test_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='binary')

validation_generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='binary')



Found 160 images belonging to 2 classes.
Found 1200 images belonging to 2 classes.
Found 1200 images belonging to 2 classes.


In [9]:
# On Re-definit les paramétres pour l'entrainement :

epochs = 4
train_samples = 160
validation_samples = 1200
test_samples = 1200

In [10]:
# Re-Entrainement du modele :

model.fit_generator(
        train_generator,
        steps_per_epoch=train_samples // batch_size,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=validation_samples// batch_size)

Epoch 1/4
5/5 [==============================] - 82s 16s/step - loss: 1.0145 - accuracy: 0.5250 - val_loss: 0.4593 - val_accuracy: 0.8336
Epoch 2/4
5/5 [==============================] - 81s 16s/step - loss: 0.6772 - accuracy: 0.6125 - val_loss: 0.5362 - val_accuracy: 0.8454
Epoch 3/4
5/5 [==============================] - 79s 16s/step - loss: 0.5051 - accuracy: 0.7750 - val_loss: 0.5807 - val_accuracy: 0.6917
Epoch 4/4
5/5 [==============================] - 84s 17s/step - loss: 0.3920 - accuracy: 0.8875 - val_loss: 0.5720 - val_accuracy: 0.7618


In [16]:
# On evalue le modele sur les donnes de validation :

resultValidation = model.evaluate_generator(validation_generator, validation_samples)

# On evalue le modele sur les donnes de test :

resultTest = model.evaluate_generator(test_generator, test_samples)

resultExemple = testImage("Data/EXEMPLE/COVID1")

    # Visualisation de l'image puis test puis affichage du resulat
    
print(resultValidation)
print(resultTest)
print(resultExemple)

NameError: name 'test_samples' is not defined

In [ ]:
# On enregistre l'historique et les poids :

model.save_weights('models/scratch_10G_V1_AUGMENTED.h5')

In [ ]:
# Comparaison :

In [ ]:
# Fin du programme ...